<a href="https://colab.research.google.com/github/kognavi/sentiment-analysis-test/blob/main/sentiment_analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip in!pip install transformers datasets tensorflow # TensorFlowを使う場合
!pip install transformers datasets tensorflow
# !pip install transformers datasets torch # PyTorchを使う場合 (今回はTensorFlowで進めます)
!pip install accelerate -U # Trainerを使う場合に推奨


ERROR: unknown command "in!pip"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 28.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5


In [5]:
!pip install --upgrade --force-reinstall scipy


  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>

In [14]:
# ステップ2: データセットの準備
from datasets import load_dataset

# IMDbデータセットをロード
imdb_dataset = load_dataset("imdb")

# データセットの構造を確認
print(imdb_dataset)

# 訓練データの一部を確認
print("\n訓練データの例:")
print(imdb_dataset["train"][0])

# 訓練用と検証用に分割
train_dataset = imdb_dataset["train"]
val_dataset = imdb_dataset["test"]

# (オプション) データ量を減らす場合
# train_dataset = imdb_dataset["train"].shuffle(seed=42).select(range(1000))
# val_dataset = imdb_dataset["test"].shuffle(seed=42).select(range(1000))


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

訓練データの例:
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking polit

In [15]:
# ステップ3: データセットの準備 (raw_datasets -> imdb_dataset 修正版)
from transformers import AutoTokenizer
import tensorflow as tf

# トークナイザのロード
model_checkpoint = "distilbert-base-uncased"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    print("トークナイザのロード完了。")
except Exception as e:
    print(f"トークナイザのロード中にエラーが発生しました: {e}")
    # エラー処理をここに記述することも可能です (例: プログラム終了)

# トークン化関数
def tokenize_function(examples):
    # "text" カラムが存在するか確認してからアクセス
    if "text" in examples:
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    else:
        # "text" カラムがない場合の処理 (エラーを出すか、空の辞書を返すなど)
        print("警告: 'text' カラムが見つかりません。空のトークン化結果を返します。")
        # 必要に応じてエラー処理を追加
        return {} # または適切なデフォルト値を返す

# データセットのトークン化
try:
    print("トークン化を開始します... (少し時間がかかる場合があります)")
    # === 修正箇所 ===
    # raw_datasets を imdb_dataset に変更
    tokenized_datasets = imdb_dataset.map(tokenize_function, batched=True)
    # ================
    print("トークン化が完了しました。")
except NameError as e:
     print(f"トークン化中に NameError が発生しました: {e}")
     print("-> 'imdb_dataset' 変数が正しくロードされているか確認してください。")
except KeyError as e:
     print(f"トークン化中に KeyError が発生しました: {e}")
     print("-> データセットに 'text' カラムが存在するか確認してください。")
except Exception as e:
    print(f"トークン化中に予期せぬエラーが発生しました: {e}")
    # その他のエラー処理

# 'tokenized_datasets' が正しく生成されたか確認してから後続処理へ
if 'tokenized_datasets' in locals():

    # 不要なカラムの削除 (より安全な方法)
    if 'text' in tokenized_datasets['train'].column_names:
        tokenized_datasets = tokenized_datasets.remove_columns(["text"])
        print("'text' カラムを削除しました。")
    else:
        print("'text' カラムは既に削除されているか、存在しません。")

    # ラベルカラムの名前を変更 (TensorFlowモデルが期待する 'labels' に)
    if 'label' in tokenized_datasets['train'].column_names:
        tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
        print("'label' カラムを 'labels' に変更しました。")
    elif 'labels' in tokenized_datasets['train'].column_names:
         print("'labels' カラムは既に存在します。")
    else:
        print("警告: 'label' または 'labels' カラムが見つかりません。")


    # データセットをTensorFlow形式に変換
    batch_size = 16 # バッチサイズ (GPUメモリに応じて調整)

    try:
        print("\nモデルの prepare_tf_dataset を使用して TF データセットを準備します...")
        tf_train_dataset = model.prepare_tf_dataset(
            tokenized_datasets["train"],
            shuffle=True,
            batch_size=batch_size,
            tokenizer=tokenizer
        )

        tf_val_dataset = model.prepare_tf_dataset(
            tokenized_datasets["test"],
            shuffle=False,
            batch_size=batch_size,
            tokenizer=tokenizer
        )
        print("データセットの準備が完了しました。")

        # データセット形式の確認
        print("\nトークン化・バッチ化後の訓練データセットの形式:")
        print(tf_train_dataset.element_spec)

        print("\nトークン化・バッチ化後の検証データセットの形式:")
        print(tf_val_dataset.element_spec)

        # 最初の訓練バッチの例を確認
        print("\n最初の訓練バッチの例:")
        for batch in tf_train_dataset.take(1):
            if isinstance(batch, tuple) and len(batch) == 2:
                features, labels = batch
                print("バッチは (features, labels) のタプル形式です。")
                if isinstance(features, dict):
                     print("  Features (辞書) のキー:", features.keys())
                     print("  Features['input_ids'] の形状:", features['input_ids'].shape)
                     print("  Features['attention_mask'] の形状:", features['attention_mask'].shape)
                else:
                     print("  Features の型:", type(features))
                print("  Labels の形状:", labels.shape)
            elif isinstance(batch, dict):
                print("バッチは辞書形式です。")
                print("  キー:", batch.keys())
                # ... (以前のコードと同様)
            else:
                print(f"予期しないバッチ形式です: {type(batch)}")

    except Exception as e:
        print(f"\nTFデータセットの準備中にエラーが発生しました: {e}")
        # エラー処理

else:
    print("エラー: 'tokenized_datasets' が定義されなかったため、処理を続行できません。")
    print("-> トークン化のステップでエラーが発生していないか確認してください。")



トークナイザのロード完了。
トークン化を開始します... (少し時間がかかる場合があります)
トークン化が完了しました。
'text' カラムを削除しました。
'label' カラムを 'labels' に変更しました。

モデルの prepare_tf_dataset を使用して TF データセットを準備します...
データセットの準備が完了しました。

トークン化・バッチ化後の訓練データセットの形式:
({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16,), dtype=tf.int64, name=None))

トークン化・バッチ化後の検証データセットの形式:
({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))

最初の訓練バッチの例:
バッチは (features, labels) のタプル形式です。
  Features (辞書) のキー: dict_keys(['input_ids', 'attention_mask'])
  Features['input_ids'] の形状: (16, 512)
  Features['attention_mask'] の形状: (16, 512)
  Labels の形状: (16,)


In [16]:
# ステップ4: モデルの準備
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf

# 事前学習済みモデルをシーケンス分類タスク用にロード
# num_labels=2: ポジティブ(1)とネガティブ(0)の2クラス分類
try:
    model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
    print("モデルのロードが完了しました。")
except Exception as e:
    print(f"モデルのロード中にエラーが発生しました: {e}")
    print("GPUメモリが不足している可能性があります。ランタイムを再起動し、バッチサイズを小さくして試してください。")
    # 必要であればここで処理を中断

# モデルの構造を確認 (オプション)
# model.summary() # Kerasモデルとして構造を表示


# 'tokenized_datasets' が正しく生成されたか確認してから後続処理へ
# (ステップ3で定義されている必要があります)
if 'tokenized_datasets' in locals():

    # データセットをTensorFlow形式に変換
    batch_size = 16 # バッチサイズ (GPUメモリに応じて調整)

    try:
        print("\nモデルの prepare_tf_dataset を使用して TF データセットを準備します...")
        tf_train_dataset = model.prepare_tf_dataset(
            tokenized_datasets["train"],
            shuffle=True,
            batch_size=batch_size,
            tokenizer=tokenizer
        )

        tf_val_dataset = model.prepare_tf_dataset(
            tokenized_datasets["test"],
            shuffle=False,
            batch_size=batch_size,
            tokenizer=tokenizer
        )
        print("データセットの準備が完了しました。")
    except Exception as e:
        print(f"\nTFデータセットの準備中にエラーが発生しました: {e}")
        # エラー処理
        # ここで適切なエラー処理を追加 (例: プログラム終了)
        raise  # エラーを再送出して処理を中断

    # オプティマイザ、損失関数、メトリクスを設定してモデルをコンパイル
    # オプティマイザ: AdamW (Adam Weight Decay) が推奨されることが多い
    # 学習率のスケジューリングを使うと、より安定した学習が期待できる
    num_train_steps = len(tf_train_dataset) # 1エポックあたりのステップ数 (tf_train_dataset が定義された後に計算)
    num_epochs = 3 # エポック数 (調整可能)

    # 学習率スケジューラ (例: 線形減衰)
    lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=5e-5, # 初期学習率 (推奨値の例)
        decay_steps=num_train_steps * num_epochs,
        end_learning_rate=0.0,
    )
    optimizer = tf.keras.optimizers.AdamW(learning_rate=lr_scheduler)

    # 損失関数: スパースカテゴリカルクロスエントロピー (ラベルが整数のため)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # from_logits=True: モデルの出力がロジット (確率に変換される前の値) であることを示す

    # メトリクス: 正確度 (Accuracy)
    metrics = ['accuracy']

    # モデルのコンパイル
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    print("\nモデルのコンパイルが完了しました。")
    print(f"オプティマイザ: {type(optimizer).__name__}")
    print(f"損失関数: {type(loss).__name__}")
    print(f"メトリクス: {metrics}")
else:
    print("エラー: 'tokenized_datasets' が定義されなかったため、モデルの準備を続行できません。")
    print("-> トークン化のステップでエラーが発生していないか確認してください。")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

モデルのロードが完了しました。

モデルの prepare_tf_dataset を使用して TF データセットを準備します...
データセットの準備が完了しました。

モデルのコンパイルが完了しました。
オプティマイザ: AdamW
損失関数: SparseCategoricalCrossentropy
メトリクス: ['accuracy']


In [17]:
# --- 設定値 ---
TRAIN_STEPS_FOR_TEST = 1  # 訓練ステップ数を1に設定
VAL_STEPS_FOR_TEST = 1    # 検証ステップ数を1に設定
EPOCHS_FOR_TEST = 1       # エポック数を1に設定
# -------------

print("モデルの訓練を開始します (最小ステップでのテスト実行)...")
print(f"訓練データセットのステップ数 (テスト用): {TRAIN_STEPS_FOR_TEST}")
print(f"検証データセットのステップ数 (テスト用): {VAL_STEPS_FOR_TEST}")
print(f"エポック数 (テスト用): {EPOCHS_FOR_TEST}")

# model変数とtf_train_dataset, tf_val_dataset変数が
# このコードの前に定義されていることを確認してください。
# 例:
# model = create_model() # あなたのモデル構築関数
# tf_train_dataset = ... # あなたの訓練データセット
# tf_val_dataset = ...   # あなたの検証データセット

try:
    # .take() を使ってデータセットから指定したステップ数だけを取り出す
    # steps_per_epoch と validation_steps も指定して、1エポックあたりのステップ数を制限する
    history = model.fit(
        tf_train_dataset.take(TRAIN_STEPS_FOR_TEST), # 訓練データを1ステップ分だけ使用
        validation_data=tf_val_dataset.take(VAL_STEPS_FOR_TEST), # 検証データを1ステップ分だけ使用
        epochs=EPOCHS_FOR_TEST,                      # 1エポックだけ実行
        steps_per_epoch=TRAIN_STEPS_FOR_TEST,        # 1エポックあたりの訓練ステップ数を1に指定
        validation_steps=VAL_STEPS_FOR_TEST          # 1エポックあたりの検証ステップ数を1に指定
    )
    print("\n最小ステップでのテスト実行 完了。")

    # 訓練履歴の確認 (オプション)
    # ステップ数が1なので、あまり意味のある履歴にはなりませんが、形式は確認できます。
    print("\n訓練履歴 (最小ステップテスト):")
    print(history.history)

except Exception as e:
    print(f"\nモデルの訓練中(最小ステップテスト)にエラーが発生しました: {e}")

# --- この後のコード (例: model.evaluate, model.save など) ---
# ... (学習はほぼ行われていないモデルで実行されることに注意) ...


モデルの訓練を開始します (最小ステップでのテスト実行)...
訓練データセットのステップ数 (テスト用): 1
検証データセットのステップ数 (テスト用): 1
エポック数 (テスト用): 1
1/1 [==============================] - 89s 89s/step - loss: 0.7011 - accuracy: 0.3750 - val_loss: 0.7601 - val_accuracy: 0.0000e+00

最小ステップでのテスト実行 完了。

訓練履歴 (最小ステップテスト):
{'loss': [0.7011189460754395], 'accuracy': [0.375], 'val_loss': [0.7601122856140137], 'val_accuracy': [0.0]}


In [18]:
# --- 設定値 ---
EVAL_STEPS_FOR_TEST = 1 # 評価ステップ数を1に設定
# -------------

print(f"\nモデルの評価を開始します (最小ステップでのテスト実行)...")
print(f"評価データセットのステップ数 (テスト用): {EVAL_STEPS_FOR_TEST}")

# model変数とtf_val_dataset変数が
# このコードの前に定義されていることを確認してください。
# 例:
# model = ... (学習済み、またはfitのテスト実行後のモデル)
# tf_val_dataset = ... (検証データセット)

try:
    # .take() を使ってデータセットから指定したステップ数だけを取り出す
    # steps 引数を指定して、評価に使用するステップ数を制限する
    results = model.evaluate(
        tf_val_dataset.take(EVAL_STEPS_FOR_TEST), # 検証データを1ステップ分だけ使用
        steps=EVAL_STEPS_FOR_TEST                 # 評価ステップ数を1に指定
    )
    print("\n最小ステップでの評価実行 完了。")

    # 評価結果の表示 (形式確認)
    # 損失と精度が表示されるはずです。値自体は学習度合いによります。
    print("\n評価結果 (最小ステップテスト):")
    print(f"Loss: {results[0]}")
    print(f"Accuracy: {results[1]}") # model.compileで指定したメトリクス名に合わせる

except Exception as e:
    print(f"\nモデルの評価中(最小ステップテスト)にエラーが発生しました: {e}")

# --- この後のコード ---
# ...



モデルの評価を開始します (最小ステップでのテスト実行)...
評価データセットのステップ数 (テスト用): 1
1/1 [==============================] - 17s 17s/step - loss: 0.7601 - accuracy: 0.0000e+00

最小ステップでの評価実行 完了。

評価結果 (最小ステップテスト):
Loss: 0.7601122856140137
Accuracy: 0.0


In [19]:
import tensorflow as tf
import numpy as np

# --- 設定値 ---
# tokenizer変数とmodel変数が定義されていることを確認
# 例:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") # または使用したモデルのTokenizer
# model = ... (学習済み、またはfit/evaluateのテスト実行後のモデル)

# テスト用の例文
test_reviews = [
    "This movie was absolutely fantastic! Highly recommend.",
    "What a waste of time, the plot was terrible.",
    "It was okay, not great but not bad either."
]

print("\n推論処理を開始します (テスト実行)...")
print("テスト用のレビュー:")
for review in test_reviews:
    print(f"- {review}")

try:
    # 1. トークン化
    #    Hugging FaceのTokenizerは通常、辞書形式でinput_idsなどを返す
    #    TensorFlowモデルに入力するために、適切な形式に変換する
    #    ここでは padding と truncation を有効にし、TensorFlowのテンソルを返すように指定
    inputs = tokenizer(
        test_reviews,
        padding=True,
        truncation=True,
        return_tensors="tf" # TensorFlow形式のテンソルで返す
    )
    print("\nトークン化 完了。入力形式:", inputs['input_ids'].shape) # 入力IDの形状を確認

    # 2. モデルによる予測
    #    モデルは通常、クラスごとの生のスコア (logits) を出力する
    outputs = model(inputs) # モデルに入力を渡す

    # Hugging FaceのTFモデルの場合、出力は TFSequenceClassifierOutput オブジェクトなどになることが多い
    # その場合、logits 属性にアクセスする
    # モデルの出力形式に合わせて調整が必要な場合があります
    if hasattr(outputs, 'logits'):
        logits = outputs.logits
    elif isinstance(outputs, tf.Tensor): # 単純なTensorで返ってくる場合
         logits = outputs
    else:
         # 不明な出力形式の場合、エラーを出すか、デバッグ情報を表示
         print("警告: モデルの出力形式が想定外です。logitsの取得に失敗しました。")
         print("モデル出力:", outputs)
         logits = None # エラーを防ぐためNoneにしておく

    if logits is not None:
        print("\nモデルによる予測 (logits) 完了。形状:", logits.shape) # Logitsの形状を確認

        # 3. 結果の解釈 (確率に変換し、クラスを決定)
        #    Logitsをソフトマックス関数で確率に変換
        probabilities = tf.nn.softmax(logits, axis=-1).numpy() # numpy配列に変換

        # 各レビューに対する予測クラス (0: ネガティブ, 1: ポジティブ と仮定)
        predicted_classes = np.argmax(probabilities, axis=-1)

        print("\n予測結果:")
        for i, review in enumerate(test_reviews):
            pred_class = predicted_classes[i]
            pred_label = "ポジティブ" if pred_class == 1 else "ネガティブ" # ラベルのマッピングが必要
            confidence = probabilities[i][pred_class] # 予測クラスの確率
            print(f"- レビュー: \"{review[:50]}...\"") # 長いレビューは省略
            print(f"  予測: {pred_label} (確率: {confidence:.4f})")
            # print(f"  (クラス0確率: {probabilities[i][0]:.4f}, クラス1確率: {probabilities[i][1]:.4f})") # 詳細表示用

    print("\n推論処理のテスト実行 完了。")

except Exception as e:
    print(f"\n推論処理中(テスト実行)にエラーが発生しました: {e}")
    import traceback
    traceback.print_exc() # 詳細なエラー情報を表示


# --- この後のコード ---
# ...



推論処理を開始します (テスト実行)...
テスト用のレビュー:
- This movie was absolutely fantastic! Highly recommend.
- What a waste of time, the plot was terrible.
- It was okay, not great but not bad either.

トークン化 完了。入力形式: (3, 13)

モデルによる予測 (logits) 完了。形状: (3, 2)

予測結果:
- レビュー: "This movie was absolutely fantastic! Highly recomm..."
  予測: ポジティブ (確率: 0.5018)
- レビュー: "What a waste of time, the plot was terrible...."
  予測: ポジティブ (確率: 0.5007)
- レビュー: "It was okay, not great but not bad either...."
  予測: ポジティブ (確率: 0.5011)

推論処理のテスト実行 完了。
